# Data Cleaning and Processing

In [2]:
import pandas as pd
import gzip

# 5-K Fashion Dataset 
    (AMAZON_FASHION_5.json)

Format is one-review-per-line in json.

- **overall**: ratings of the product
- **reviewerID**: ID of the reviewer, e.g. A2SUAM1J3GNN3B
- **asin**: ID of the product, e.g. 0000013714
- **reviewerName**: name of the reviewer
- **vote**: helpful votes of the review
- **style**: a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- **reviewText**: text of the review
- **summary**: summary of the review
- **unixReviewTime**: time of the review (unix time)
- **reviewTime**: time of the review (raw)
- **image**: images that users post after they have received the product



In [31]:
# data path
fashion_data = "/Users/pavansingh/Library/CloudStorage/GoogleDrive-sngpav003@myuct.ac.za/My Drive/Masters 2022/Dissertation/Masters-Dissertation/Data/AMAZON_FASHION_5.json"
fashion_data = pd.read_json(fashion_data, lines=True)
fashion_data = fashion_data.loc[:,['reviewerName', 'reviewText', 'overall', 'style']]
display(fashion_data.loc[10:20,:])
print("Shape of Data:", fashion_data.shape)

,reviewerName,reviewText,overall,style
10,Denise A. Conte,Relieved my Plantar Fascitis for 3 Days. Then ...,2,"{'Size Name:': ' Men's 6-6.5, Women's 8-8.5', ..."
11,Cognizant Consumer,This is my 6th pair and they are the best thin...,2,"{'Size Name:': ' Men's 8-8.5, Women's 10-10.5'..."
12,William_Jasper,We have used these inserts for years. They pr...,5,NaN
13,Cognizant Consumer,Pinnacle seems to have more cushioning so my h...,5,NaN
14,William_Jasper,Excellent insole with good support.,5,NaN
15,Cognizant Consumer,A little more cushion than the Powerstep Prote...,5,NaN
16,William_Jasper,These insoles help my heels feel much better. ...,5,NaN
17,Denise A. Conte,Relieved my Plantar Fascitis for 3 Days. Then ...,2,NaN
18,Cognizant Consumer,This is my 6th pair and they are the best thin...,2,NaN
19,William_Jasper,We have used these inserts for years. They pr...,5,NaN


Shape of Data: (3176, 4)


In [37]:
fashion_data['style'].isna().sum()
fashion_data.dropna(subset=["style"], inplace=True)
fashion_data.sort_values()
display(fashion_data.head(10))
print("Shape of Data:", fashion_data.shape)

,reviewerName,reviewText,overall,style
0,Tonya B.,Great product and price!,5,"{'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}"
1,Tonya B.,Great product and price!,5,"{'Size:': ' Big Boys', 'Color:': ' Black (3746..."
2,Tonya B.,Great product and price!,5,"{'Size:': ' Big Boys', 'Color:': ' Blue/Gray L..."
3,Tonya B.,Great product and price!,5,"{'Size:': ' Big Boys', 'Color:': ' Blue (37867..."
4,Tonya B.,Great product and price!,5,"{'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}"
5,NaeNae,Waaay too small. Will use for futur children!,3,"{'Size:': ' Little Boys', 'Color:': ' White/Bl..."
6,NaeNae,Stays vibrant after many washes,5,"{'Size:': ' Little Boys', 'Color:': ' Blue/Ora..."
7,NaeNae,Stays vibrant after many washes,5,"{'Size:': ' Little Boys', 'Color:': ' Blue (37..."
8,NaeNae,My son really likes the pink. Ones which I was...,5,"{'Size:': ' Little Boys', 'Color:': ' Blue/Pink'}"
9,NaeNae,Waaay too small. Will use for future child.,3,"{'Size:': ' Little Boys', 'Color:': ' Light Bl..."


Shape of Data: (3107, 4)
